In [1]:
from radar_fusion import fuse_5_frames
from vod.configuration import KittiLocations
from vod.frame import FrameTransformMatrix, transform_pcl, FrameDataLoader
import numpy as np
import k3d
from utils import read_kitti_label_file, plot_moving_objs_bbox
kitti_locations = KittiLocations(root_dir='/datasets/vod')
print('Radar data path:', kitti_locations.radar_dir)

Radar data path: /datasets/vod/radar/training/velodyne


In [6]:
frame_number = 3629
frame_data = FrameDataLoader(kitti_locations=kitti_locations, frame_number=str(frame_number).zfill(5))
transforms = FrameTransformMatrix(frame_data)
_, moving_objs = read_kitti_label_file(frame_data.raw_labels)
lidar_points_cf = transform_pcl(frame_data.lidar_data, transforms.t_camera_lidar)[:,:3]

my_fused = np.fromfile(f'/datasets/vod/my_radar_5frames/training/velodyne/{str(frame_number).zfill(5)}.bin', dtype=np.float32).reshape(-1, 7) # 雷达坐标系
my_fused_cf = transform_pcl(my_fused, transforms.t_camera_radar)[:,:3] # 转换到相机坐标系
original_fused_radar_pc = np.fromfile(f'/datasets/vod/radar_5frames/training/velodyne/{str(frame_number).zfill(5)}.bin', dtype=np.float32).reshape(-1, 7) # 雷达坐标系
original_cf_fused_radar_pc = transform_pcl(original_fused_radar_pc, transforms.t_camera_radar)[:,:3]

print(original_fused_radar_pc.shape)
print(my_fused.shape)


(1620, 7)
(1620, 7)


In [7]:
plot = k3d.plot(camera_auto_fit=False, axes_helper=True, grid_visible=False)
plot_moving_objs_bbox(plot, moving_objs, transforms.t_lidar_camera, transforms.t_camera_lidar)
plot += k3d.points(np.array([0,0,0]), point_size=0.3, color=0x000000)
plot += k3d.points(lidar_points_cf, point_size=0.05, color=0x000000)

plot += k3d.points(my_fused_cf, point_size=0.3, color=0xff0000) #红色, 自行融合后的雷达点云
plot += k3d.points(original_cf_fused_radar_pc, point_size=0.3, color=0x0000ff) # 蓝色，原始雷达点云

plot.display()

Output()